# 기계학습개론 과제1
응용통계학과 
3219****
이동현

In [321]:
import pandas as pd
import numpy as np
import tensorflow as tf
import sklearn
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.metrics import confusion_matrix, roc_auc_score, accuracy_score, f1_score, precision_recall_curve
from sklearn.metrics import roc_curve, average_precision_score, matthews_corrcoef
from sklearn.preprocessing import StandardScaler


In [322]:
path = "C://Users//3219****//Desktop//교과//4학년 2학기//기계학습이론//"
df = pd.read_csv(path + "PrOCTOR_sample_data_all.csv")
df.set_index("Drug", inplace=True)
df

,Label,MolecularWeight,XLogP,HydrogenBondDonorCount,HydrogenBondAcceptorCount,PolarSurfaceArea,FormalCharge,NumRings,RotatableBondCount,Refractivity,...,Prostate,Spleen,Liver,Fallopian Tube,Ovary,Breast,Bladder,Cervix Uteri,Small Intestine,Salivary Gland
Drug,,,,,,,,,,,,,,,,,,,,,
Abiraterone,passed,349.240565,3.965293,1,2,33.12,0,5.0,1,107.3024,...,0.804379,0.734742,1.722980,0.994414,0.943392,0.736495,0.780652,0.809155,0.377117,0.300038
Acebutolol,passed,336.204907,1.534682,3,5,87.66,0,1.0,10,94.8692,...,4.420117,6.518067,2.618737,2.319843,0.295605,10.000000,2.251212,3.500741,2.735411,5.831739
Acenocoumarol,passed,353.089937,2.684692,1,5,109.42,0,3.0,5,94.1848,...,0.282973,0.368951,0.988317,0.426425,0.430046,0.356781,0.478249,0.500222,0.492640,0.357384
Acepromazine,passed,326.145284,3.488603,0,3,23.55,0,3.0,5,99.3543,...,1.126801,7.962256,7.432229,3.192496,1.602348,1.393756,0.529358,0.387122,0.974304,6.041281
Aceprometazine,passed,326.145284,3.845218,0,3,23.55,0,3.0,4,98.9077,...,1.168346,0.453772,0.050176,2.241176,1.707206,2.094832,2.018864,2.547429,1.588867,0.826448
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zidovudine,passed,267.096754,-0.298709,2,6,108.30,0,2.0,3,61.7045,...,0.007795,0.212996,0.000000,0.012702,0.000000,0.002419,0.000000,0.005181,0.521985,0.053856
Ziprasidone,passed,412.112460,4.304046,1,4,48.47,0,5.0,4,116.7248,...,10.000000,10.000000,7.432229,10.000000,10.000000,10.000000,5.551624,10.000000,4.168484,6.041281
Zoledronate,passed,271.996324,-3.851109,5,8,153.11,0,1.0,4,52.1577,...,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000


In [323]:
print("--------데이터셋 차원 확인")
print(df.shape)
print("--------결측값 있는 변수 확인")
missval = df.isnull().sum()[df.isnull().sum()!=0]
print(missval)
missval_name = missval.index #결측값 있는 변수 이름 벡터로 추출


--------데이터셋 차원 확인
(828, 48)
--------결측값 있는 변수 확인
NumRings                1
LogpSolubility         21
Ro5                     1
Ghose                   1
Veber                   1
ExAC_loss_frequency    31
dtype: int64


### 결측값 대체 - 중앙값

1 -1) 중앙값으로 결측값을 대체

In [324]:
df_med = df.copy()
for i in range(0,len(missval_name)):
    df_med[missval_name[i]] = df_med[missval_name[i]].replace(np.NaN, df_med[missval_name[i]].median())
    print(missval_name[i],"변수의 결측값 :", df_med[missval_name[i]].isnull().sum())

NumRings 변수의 결측값 : 0
LogpSolubility 변수의 결측값 : 0
Ro5 변수의 결측값 : 0
Ghose 변수의 결측값 : 0
Veber 변수의 결측값 : 0
ExAC_loss_frequency 변수의 결측값 : 0


1 - 2) 중앙값 대체에 대한 SGD Classifier 분류

In [325]:
df_med["Label"].replace({'failed':0, 'passed':1}, inplace=True) #문자 범주형 변수를 숫자로 dummy화 함
x1 = df_med.drop("Label", axis=1)
y1 = df_med["Label"]
x1 = np.array(x1)

In [326]:
kfold_5 = KFold(n_splits = 5, shuffle=False)
for train_index, test_index in kfold_5.split(x1):
    X1_train, X1_test = x1[train_index], x1[test_index]
    y1_train, y1_test = y1[train_index], y1[test_index]

In [327]:
scaler = StandardScaler()
scaler.fit(X1_train)
X1_train = scaler.transform(X1_train)
X1_test = scaler.transform(X1_test)

In [328]:
model1 = SGDClassifier(loss='log')
model1.fit(X1_train,y1_train)
y1_pred = model1.predict(X1_test)

C:\Users\32197453\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:173: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(


1 - 3) Accuracy, AUC, AUPRC, F1-Score, Matthews correlation coefficient (MCC)

In [329]:
print(confusion_matrix(y1_test, y1_pred)); print()

#Accuracy
print("Accuracy :", accuracy_score(y1_test, y1_pred))

#AUC
print("AUC :", roc_auc_score(y1_test, y1_pred))

#AUPRC
print("AUPRC :", average_precision_score(y1_test, y1_pred))

#F1-score
print("F1-Score :", f1_score(y1_test, y1_pred))

#MCC
print( "MCC :", matthews_corrcoef(y1_test, y1_pred))

[[  2  17]
 [ 14 132]]

Accuracy : 0.8121212121212121
AUC : 0.5046863734679163
AUPRC : 0.8858046308446235
F1-Score : 0.894915254237288
MCC : 0.010110368712774173


### 결측값 대체 - MICE

2-1) MICE로 결측값을 대체

In [330]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [331]:
df_mice = df.copy()
imputer = IterativeImputer(random_state=7453)
df_mice[missval_name] = imputer.fit_transform(df_mice[missval_name])

print("변수의 결측값 :","\n",df_mice[missval_name].isnull().sum())

변수의 결측값 : 
 NumRings               0
LogpSolubility         0
Ro5                    0
Ghose                  0
Veber                  0
ExAC_loss_frequency    0
dtype: int64


2 - 2) MICE 대체에 대한 SGD Classifier 분류

In [332]:
df_mice["Label"].replace({'failed':0, 'passed':1}, inplace=True) #문자 범주형 변수를 숫자로 dummy화 함
x2 = df_mice.drop("Label", axis=1)
y2 = df_mice["Label"]
x2 = np.array(x2)

In [333]:
kfold_5 = KFold(n_splits = 5, shuffle=False)
for train_index, test_index in kfold_5.split(x2):
    X2_train, X2_test = x2[train_index], x2[test_index]
    y2_train, y2_test = y2[train_index], y2[test_index]

In [334]:
scaler = StandardScaler()
scaler.fit(X2_train)
X2_train = scaler.transform(X2_train)
X2_test = scaler.transform(X2_test)

In [335]:
model2 = SGDClassifier(loss='log')
model2.fit(X2_train,y2_train)
y2_pred = model2.predict(X2_test)

C:\Users\32197453\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:173: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(


2 - 3) Accuracy, AUC, AUPRC, F1-Score, Matthews correlation coefficient (MCC)

In [336]:
print(confusion_matrix(y2_test, y2_pred)); print()

#Accuracy
print("Accuracy :", accuracy_score(y2_test, y2_pred))

#AUC
print("AUC :", roc_auc_score(y2_test, y2_pred))

#AUPRC
print("AUPRC :", average_precision_score(y2_test, y2_pred))

#F1-score
print("F1-Score :", f1_score(y2_test, y2_pred))

#MCC
print( "MCC :", matthews_corrcoef(y2_test, y2_pred))

[[  6  13]
 [ 13 133]]

Accuracy : 0.8424242424242424
AUC : 0.6133741888968998
AUPRC : 0.9086340037644223
F1-Score : 0.910958904109589
MCC : 0.22674837779379955


### 결론


데이터가 변수에 따라 단위달라 값의 범위가 달라서 Scaling을 하지 않고 Classification을 하면 TN과 FP가 0으로 나타난다. 또한 이에 따라 AUC와 MCC값이 각각 0.5, 0으로 나타났기 때문에 Standard Scaling을 실행했다. 그 결과 결측치를 중앙값으로 대체한 것보다 MICE로 대체한 것이 5가지 성능평가지표에서 모두 더 높은 값을 나타낸다.